In [25]:
import pandas as pd
from bs4 import BeautifulSoup 
import re
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /home/alex/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alex/nltk_data...


In [11]:
df = pd.read_csv("twitter_training.csv", header=None)

In [12]:
# find and delete null values
df.dropna(inplace=True)

In [14]:
df[2].value_counts()

Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: 2, dtype: int64

In [16]:
# As we want our model to just predict positive, negative oder neurtral, let us delete irrelevant entries in our data
data = df[df[2]!='Irrelevant']

In [18]:
# we also delete the first to column as they are irrelevant for us
data.drop(columns=[0,1],inplace=True)

/tmp/ipykernel_364440/2552913855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=[0,1],inplace=True)


,2,3
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


# As our Text are tweets wie habe to clean by:

* removing all html Tags
* remove all url in the sentences
* remove all specials characters(emogis,...)
* remove stopwords because they are irrelevant for BERT

In [26]:

# removing not and no from stop words as we need them for our model training
stopwords = stopwords.words("english")
stopwords.remove('not')
stopwords.remove('no')

# intializing method for lemmatizing words
lemmatizer = WordNetLemmatizer()

# now creating funtion to clean our data
def clean_text(text):
    # remove any html tags
    new_text = BeautifulSoup(text).get_text()
    
    # remove urls from reviews
    no_urls = new_text.replace('http\S+', '').replace('www\S+', '')
    
    # remove any non-letters
    clean_text = re.sub("[^a-zA-Z]", " ", no_urls)
    
    # convert whole sentence to lowercase and split
    new_words = clean_text.lower().split()
    
    # converting stopwords list to set for faster search
    stops = set(stopwords)
    
    # using stopwords to remove irrelavent words and lemmatizing the final output
    final_words = [lemmatizer.lemmatize(word) for word in new_words if not word in stops]
    
    # return the final result
    return (" ".join(final_words))

# apply the clean method to ours tweets
data["text"] = data[3].apply(lambda x:clean_text(x))

/tmp/ipykernel_364440/3949118108.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  new_text = BeautifulSoup(text).get_text()
/tmp/ipykernel_364440/3949118108.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data[3].apply(lambda x:clean_text(x))


In [28]:
# delete the all tweer from the dataset
data.drop(columns=[3],inplace=True)

/tmp/ipykernel_364440/4268734947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=[3],inplace=True)


In [33]:
# delete all colum with text < 10 charachter
data = data[data["text"].apply(lambda x: len(x) > 0)]
data

,2,text
0,Positive,im getting borderland murder
1,Positive,coming border kill
2,Positive,im getting borderland kill
3,Positive,im coming borderland murder
4,Positive,im getting borderland murder
...,...,...
74677,Positive,realized window partition mac like year behind...
74678,Positive,realized mac window partition year behind nvid...
74679,Positive,realized window partition mac year behind nvid...
74680,Positive,realized window partition mac like year behind...


In [34]:
# save the cleaned data

data.to_csv("clean_data.csv", header=None, index=None)